In [ ]:
from sklearn.datasets import load_svmlight_file

data_path = 'svmguide3'
data = load_svmlight_file(f'../datasets/{data_path}.txt')
x = data[0].todense()
y = data[1].reshape(-1,1)

In [ ]:
import numpy as np
x = (x - np.mean(x, axis=0)) / np.std(x, axis=0)
x[np.isnan(x)] = 0
y = (y - np.mean(y, axis=0)) / np.std(y, axis=0)

In [ ]:
from scipy.linalg import lstsq
import numpy as np
k = 8
m, n = x.shape
print(m, n)
population = [np.zeros((n))]
popSize = 1
fitness = [np.zeros(2)]
fitness[0][0] = np.inf
fitness[0][1] = population[0].sum()
T = round(n*k*k*2*np.exp(1))
epoch = T // 100

tt = []
mse = []
r2 = []

for _ in range(T):
    if _ and _ % epoch == 0:
        m_fitness, m_population = np.array(fitness), np.array(population)
        temp = m_fitness[:,1]<=k
        j = max(m_fitness[temp,1])
        seq = m_fitness[:,1] == j
        selectedVariables = m_population[seq,:]
        for selectedVariable in selectedVariables:
            pos = selectedVariable == 1
            coef = lstsq(x[:,pos], y)[0]
            err = x[:,pos] * coef - y
            loss = err.T * err / m
            mse.append(loss)
            r2.append((np.var(y) - loss) / np.var(y))
        tt.append(_)
        
    idx = np.random.randint(popSize,size=1)[0]
    pm = np.random.choice((0,1), p=(1-1/n, 1/n), size=n)
    flip = 1 - population[idx]
    offspring = np.where(pm, flip, population[idx])
    offspringFit = np.zeros(2)
    offspringFit[1] = offspring.sum()
    if offspringFit[1] == 0 or offspringFit[1] >= 2*k:
        offspringFit[0] = np.inf
    else:
        pos = offspring == 1
        coef = lstsq(x[:,pos], y)[0]
        err = x[:,pos] * coef - y
        offspringFit[0] = err.T * err / m
    
    m_fitness, m_population = np.array(fitness), np.array(population)
    if sum((m_fitness[:,0]<offspringFit[0])*(m_fitness[:,1]<=offspringFit[1]))+\
        sum((m_fitness[:,0]<=offspringFit[0])*(m_fitness[:,1]<offspringFit[1]))>0:
        continue
    else:
        deleteIndex = np.array((m_fitness[:,0]>=offspringFit[0])*(m_fitness[:,1]>=offspringFit[1]))
    ndelete = ~deleteIndex
    population = list(m_population[ndelete]) + [offspring]
    fitness = list(m_fitness[ndelete]) + [offspringFit]
    popSize = len(population)
    
    
m_fitness, m_population = np.array(fitness), np.array(population)
temp = m_fitness[:,1]<=k
j = max(m_fitness[temp,1])
seq = m_fitness[:,1] == j
selectedVariables = m_population[seq,:]

In [ ]:
m_fitness, m_population = np.array(fitness), np.array(population)
temp = m_fitness[:,1]<=k
j = max(m_fitness[temp,1])
seq = m_fitness[:,1] == j
selectedVariables = m_population[seq,:]

In [ ]:
print(j)
print(m_fitness[m_fitness[:,1]<=k])

In [ ]:
for selectedVariable in selectedVariables:
    pos = selectedVariable == 1
    coef = lstsq(x[:,pos], y)[0]
    err = x[:,pos] * coef - y
    loss = err.T * err / m
    print(loss)
    print((np.var(y) - loss) / np.var(y))

In [ ]:
import matplotlib.pyplot as plt
plt.style.use(['science'])

mse = np.array(mse)
mse = [_[0][0] for _ in mse]

with plt.style.context(['science']):
    plt.figure()
    plt.plot(tt, mse)
    plt.xlabel('epoch')
    plt.ylabel('mse')
    plt.title(f'poss: {data_path}')
    plt.savefig(f'POSS_{data_path}.pdf')

In [ ]:
with plt.style.context(['science']):
    ep = m_fitness[m_fitness[:,1]<=2*k].T
    plt.scatter(ep[1], ep[0])
    plt.xlabel('k')
    plt.ylabel('mse')
    plt.title(f'Pareto front- poss: {data_path}')
    plt.savefig(f'POSS_{data_path}_pareto.pdf', bbox_inches='tight')

In [ ]:
np.save(f'POSS_{data_path}_t.npy', tt)
np.save(f'POSS_{data_path}_mse.npy', mse)
np.save(f'POSS_{data_path}_pareto.npy', ep)